In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import geopandas as gpd
from shapely.geometry import Point, Polygon

import zipfile
import requests
import os
import shutil

import tensorflow

%matplotlib inline

In [2]:
#Get data
filestring = './data/anc_out.csv'
df = pd.read_csv(filestring)

df = df.reset_index()

print(df.shape)
df['y']= df['countIssued'].shift(-80)

df = df.dropna()
df = df.drop('index_left', axis= 1)
print(df.shape)
df = df.fillna(0)
df = df.astype('float')
print(df.shape)

data = df #copy it


X = data.drop(['Util_Indx_BBL','index','y'], axis=1)
y = data['y']

assert np.all(np.isfinite(X))
assert np.all(np.isfinite(y))
assert not np.any(np.isnan(X))
assert not np.any(np.isnan(y))

(2400, 59)
(2320, 59)
(2320, 59)


In [3]:
df.dtypes;

In [4]:
#Flexible adaptation of Dr. Braman's interactive gridsearch script
#implementation. 
#TODO Clean up and streamline
import sklearn
from sklearn.neural_network import *
from sklearn.neighbors import *
from sklearn.svm import *
from sklearn.gaussian_process import *
from sklearn.gaussian_process.kernels import *
from sklearn.tree import *
from sklearn.ensemble import *
from sklearn.naive_bayes import *
from sklearn.discriminant_analysis import *
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
import random


data = df
data = data.reset_index()

#Make a short list of random states to insert into randomstate params.
scrambler = []
for scram in range(0, 10):
    scrambler.append(random.randint(0, 10000))   
print(scrambler)

to_scale = []

names       = ['AdaBoostRegressor',
             'RandomForestRegressor',
             #'SVR'
             #'KNeighborsRegressor',
             #'BaggingRegressor',
             #'GradientBoostingRegressor',
             #'LinearRegression',
             #'MLPRegressor',
             #'SGDRegressor',
             #'LassoLarsCV'         
    
]

regressors = [AdaBoostRegressor(),
              RandomForestRegressor(),
              #SVR()
              #KNeighborsRegressor(),
              #BaggingRegressor(),
              #GradientBoostingRegressor(),
              #LinearRegression(),
              #MLPRegressor(),
              #SGDRegressor(),
              #LassoLarsCV()
    
]

param_grids =[ 
    ['AdaBoostRegressor', dict(
        n_estimators=[80, 60, 30],
        learning_rate=[1, .5, .01],
        loss=['linear', 'square', 'exponential'],
        random_state=scrambler[3:5]
        
    )],
        
    ['RandomForestRegressor', dict(
        max_depth=[5, 10],
        #criterion=['mse', 'mae'],
        random_state=scrambler[:2]
    )],
    ]

[6978, 8884, 1293, 7150, 6400, 7147, 1711, 2767, 6735, 4455]


In [5]:
outcomes = []
for name, rgsr in zip(names, regressors):
    
    for item in param_grids:
        if item[0]==name:
            params= item[1]
        
    
    cv = sklearn.model_selection.GridSearchCV(rgsr, param_grid=params,
                                              verbose=True, n_jobs=12,
                                              cv=5, pre_dispatch="2*n_jobs")
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
        X, y, random_state=random.randint(0, 10000))
    fitted = cv.fit(X_train, y_train)
    score = cv.score(X_test, y_test)
    outcomes.append((name, score, cv.cv_results_))
    print(name, ' is done!')
    print(name, ' scored:', score)
    
for nm in range(0, len(outcomes)):
    print()
    print(outcomes[nm][0])
    print(outcomes[nm][1])
    print()
    
    

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    5.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:   32.2s
[Parallel(n_jobs=12)]: Done 270 out of 270 | elapsed:   51.2s finished


AdaBoostRegressor  is done!
AdaBoostRegressor  scored: 0.739829267667
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=12)]: Done  18 out of  20 | elapsed:    1.0s remaining:    0.1s
[Parallel(n_jobs=12)]: Done  20 out of  20 | elapsed:    1.0s finished


RandomForestRegressor  is done!
RandomForestRegressor  scored: 0.830613404737

AdaBoostRegressor
0.739829267667


RandomForestRegressor
0.830613404737



In [ ]:
outcomes[1][2]